# Flight assignment

A small airlane company produces a weekly plan for the assignment of pilots and co-pilots to flights.
In a pre-processing phase, the list of flights has been analysed to identify incompatible flights (those that cannot have the same crew members, like flights departing at the same hour). 

Our company follows these rules:
- each flight needs a pilot and a co-pilot to depart. 
- both pilots and co-pilots cannot exceed a weekly amount of flight hours.
- each pilot (resp. co-pilot) is paid based on several factors (e.g., the experience, the rank, flight hours)

The airplane company what to produce a weekly plan that minimizes the costs.

## Data

$P:$ set of pilots  
$C:$ set of co-pilots  
$V:$ set of flights  
$I_v \subseteq V :$ flights incompatibilities of flight $v \in V$  
$f_{pv}:$ cost per hour of pilot $p\in P$ if assigned to flight $v \in V$  
$g_{cv}:$ cost per hour of co-pilot $c\in C$ if assigned to flight $v \in V$  
$q:$ weekly amount of flight hours for each pilot and co-pilot
$t_v:$ flight hours of flight $v\in V$

## Variables

$x_{pv}:$ $1$ if pilot $p$ is assigned to flight $v$, and $0$ otherwise  
$y_{cv}:$ $1$ if co-pilot $c$ is assigned to flight $v$, and $0$ otherwise  

## Objective function

Minimize flight costs
$$z^* = \min \sum_{v \in V} t_v \cdot ( \sum_{p \in P} f_{pv} \cdot x_{pv} + \sum_{c \in C} g_{cv} \cdot y_{cv} ) $$


## Constraints

One pilot and one co-pilot for each flight
$$ \sum_{p \in P} x_{pv} = 1, \forall v \in V $$  
$$ \sum_{c \in C} y_{cv} = 1, \forall v \in V $$  

Do not exceed flight hours for each pilot and co-pilot
$$ \sum_{v \in V} t_v \cdot x_{pv} \le q, \forall p \in P $$  
$$ \sum_{v \in V} t_v \cdot y_{cv} \le q, \forall c \in C $$

Avoid to assign the same crew to two incompatible flights
$$ x_{pv} + x_{pv'} \le 1, \forall p\in P, v \in V, v' \in I_v $$
$$ y_{cv} + y_{cv'} \le 1, \forall c\in C, v \in V, v' \in I_v $$

## Model implementation

Data are stored into three `csv` files, one for the crew members, one for the flights, and one for the costs.

In [1]:
import pandas as pd

crew = pd.read_csv("crew.csv")
flights = pd.read_csv("flights.csv")
costs = pd.read_csv("costs.csv", index_col=0)

In [2]:
crew[:5]

,Pilots,Co-pilots
0,Adriana C. Ocampo Uria,Cecilia Payne-Gaposchkin
1,Albert Einstein,Chien-Shiung Wu
2,Anna K. Behrensmeyer,Dorothy Hodgkin
3,Blaise Pascal,Edmond Halley
4,Caroline Herschel,Edwin Powell Hubble


In [3]:
flights[:5]

,Flights,Time,ZT2898,ZT7947,ZT1277,ZT7456,ZT3554,ZT3234,ZT3821,ZT4311,ZT7579,ZT6742,ZT1557,ZT9968,ZT5269,ZT5845,ZT6743
0,ZT2898,3.0,ZT9968,ZT7456,ZT5845,ZT2898,ZT7947,ZT9968,ZT7579,ZT9968,ZT2898,ZT2898,ZT7947,ZT2898,ZT7947,ZT2898,ZT7947
1,ZT7947,1.0,ZT7947,ZT6743,ZT3821,ZT7579,ZT1277,ZT7579,ZT6742,ZT7947,ZT6742,ZT5845,ZT1277,ZT7947,ZT1277,ZT6743,ZT1277
2,ZT1277,1.5,ZT7456,ZT5269,ZT1557,ZT6743,ZT6743,ZT3821,ZT5269,ZT1277,ZT5845,ZT1277,ZT5269,ZT1277,ZT7456,ZT1277,ZT7456
3,ZT7456,4.0,ZT6742,ZT4311,NaN,ZT4311,ZT3234,ZT7456,ZT7456,ZT6743,ZT7456,ZT7456,ZT3554,NaN,NaN,ZT7456,ZT3554
4,ZT3554,2.0,ZT4311,ZT3554,NaN,ZT3554,NaN,ZT3554,ZT3554,ZT3554,ZT3554,NaN,NaN,NaN,NaN,NaN,ZT3234


In [4]:
costs[:5]

,ZT2898,ZT7947,ZT1277,ZT7456,ZT3554,ZT3234,ZT3821,ZT4311,ZT7579,ZT6742,ZT1557,ZT9968,ZT5269,ZT5845,ZT6743
Pilots,,,,,,,,,,,,,,,
Adriana C. Ocampo Uria,42,49,46,42,49,34,32,41,45,44,43,42,49,37,47
Albert Einstein,38,48,48,46,35,44,47,43,31,42,40,47,47,33,49
Anna K. Behrensmeyer,34,43,42,36,30,30,39,42,44,48,48,36,41,42,39
Blaise Pascal,45,44,50,49,46,32,50,32,37,37,48,39,38,33,43
Caroline Herschel,49,50,50,30,42,32,30,49,47,44,30,46,31,44,48


### Data - Initializing sets

We create the abstract model and the sets of pilots $P$, co-pilots $C$, and flights $V$.

In [5]:
import pyomo.environ as pyopt

model = pyopt.AbstractModel()

model.P = pyopt.Set(initialize=crew['Pilots'].dropna().tolist())
model.C = pyopt.Set(initialize=crew['Co-pilots'].dropna().tolist())
model.V = pyopt.Set(initialize=flights['Flights'].tolist())

### Data - Flights parameters

We then read parameter $t$ for each flight, and the set $I$ of incompatibilities between flights:

In [6]:
model.t = pyopt.Param(model.V, initialize={r['Flights']: r['Time'] for i, r in flights.iterrows()})
model.I = pyopt.Param(model.V, model.V, rule=lambda model, v1, v2: 1 if v1 in flights[v2].tolist() else 0)

### Data - Pilots parameters

We then create parameters for both pilots and co-pilots costs ($f$ and $g$, resp.) and the maximum amount of weekly flying hours $q$.

In [7]:
def read_costs(model, p, v):
    return costs.at[p, v]

model.f = pyopt.Param(model.P, model.V, rule=read_costs)
model.g = pyopt.Param(model.C, model.V, rule=read_costs)
model.q = pyopt.Param(initialize=10)

### Variables

All variables are binary ones:  
$x_{pv}:$ $1$ if pilot $p$ is assigned to flight $v$, and $0$ otherwise  
$y_{cv}:$ $1$ if co-pilot $c$ is assigned to flight $v$, and $0$ otherwise  

In [8]:
model.x = pyopt.Var(model.P, model.V, within=pyopt.Binary)
model.y = pyopt.Var(model.C, model.V, within=pyopt.Binary)

### Objective function

The goal is to minimize the flight costs by mininimizing the cost of pilot and co-pilot:
$$z^* = \min \sum_{v \in V} t_v \cdot ( \sum_{p \in P} f_{pv} \cdot x_{pv} + \sum_{c \in C} g_{cv} \cdot y_{cv} ) $$


In [9]:
def objective_function(model):
    return pyopt.sum(model.t[v] * \
                (pyopt.sum(model.f[p,v] * model.x[p,v] for p in model.P) + \
                 pyopt.sum(model.g[c,v] * model.y[c,v] for c in model.C )) for v in model.V)

model.z = pyopt.Objective(rule=objective_function, sense=pyopt.minimize)

### Constraints - Assignments

We force to select a pilot and a co-pilot for each flight:
$$ \sum_{p \in P} x_{pv} = 1, \forall v \in V $$  
$$ \sum_{c \in C} y_{cv} = 1, \forall v \in V $$  

In [10]:
def pilot_assignment_cons(model, v):
    return pyopt.sum(model.x[p,v] for p in model.P) == 1

def copilot_assignment_cons(model, v):
    return pyopt.sum(model.y[c,v] for c in model.C) == 1

model.pilot_assignment = pyopt.Constraint(model.V, rule=pilot_assignment_cons)
model.copilot_assignment = pyopt.Constraint(model.V, rule=copilot_assignment_cons)

### Constraints - Flights incompatiilities

We forbid a crew member to be on two incompatible flights:
$$ x_{pv} + x_{pv'} \le 1, \forall p\in P, v \in V, v' \in I_v $$
$$ y_{cv} + y_{cv'} \le 1, \forall c\in C, v \in V, v' \in I_v $$

In [11]:
def pilot_incompatibility_cons(model, p, v1, v2):
    return (model.x[p,v1] + model.x[p,v2] <= 2 - model.I[v1,v2])

def copilot_incompatibility_cons(model, c, v1, v2):
    return (model.y[c,v1] + model.y[c,v2] <= 2 - model.I[v1,v2])

model.pilot_incompatibilities = pyopt.Constraint(model.P, model.V, model.V, rule=pilot_incompatibility_cons)
model.copilot_incompatibilities = pyopt.Constraint(model.C, model.V, model.V, rule=copilot_incompatibility_cons)

### Constraints - Fliying hours

We do not exceet pilots and co-pilots $q$ flying hours:
$$ \sum_{v \in V} t_v \cdot x_{pv} \le q, \forall p \in P $$  
$$ \sum_{v \in V} t_v \cdot y_{cv} \le q, \forall c \in C $$

In [12]:
def pilot_hours_cons(model, p):
    return pyopt.sum(model.t[v] * model.x[p,v] for v in model.V) <= model.q

def copilot_hours_cons(model, c):
    return pyopt.sum(model.t[v] * model.y[c,v] for v in model.V) <= model.q

model.pilot_hours = pyopt.Constraint(model.P, rule=pilot_hours_cons)
model.copilot_hours = pyopt.Constraint(model.C, rule=copilot_hours_cons)

## Solving the model

Once the abstract model is created, we populate the instance and solve the problem:

In [13]:
instance = model.create_instance()
solver = pyopt.SolverFactory('glpk')
results = solver.solve(instance)
print("Weekly plan cost {}".format(pyopt.value(instance.z)))

Weekly plan cost 1630.5


In [14]:
print(results)


Problem: 
- Name: unknown
  Lower bound: 1630.5
  Upper bound: 1630.5
  Number of objectives: 1
  Number of constraints: 11331
  Number of variables: 751
  Number of nonzeros: 23251
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.07746267318725586
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



We finally print the crew member assigned to each flight:

In [15]:
pd.DataFrame.from_items(
    [(v, [p,c]) for p in instance.P for c in instance.C for v in instance.V 
                if instance.x[p,v].value > 0 and instance.y[c,v].value > 0 ], 
      orient='index', columns=['Pilot', 'Co-pilot'])

,Pilot,Co-pilot
ZT6743,Sir Isaac Newton,Flossie Wong-Staal
ZT2898,Lord Kelvin,Chien-Shiung Wu
ZT3821,Lord Kelvin,Chien-Shiung Wu
ZT5269,Marie Curie,Cecilia Payne-Gaposchkin
ZT6742,Sir Ernest Rutherford,Shirley Ann Jackson
ZT1557,Jacqueline K. Barton,Rita Levi-Montalcini
ZT7579,Jocelyn Bell Burnell,Wilhelm Conrad Roentgen
ZT9968,Jocelyn Bell Burnell,Melissa Franklin
ZT5845,Johannes Kepler,Mildred S. Dresselhaus
ZT7947,Geraldine Seydoux,Niels Bohr


## Workers go on strike!

Labor unions declared a last minute strike, and some of the pilots and co-pilots of our company joined the strike.

The previous plan must be then updated to recover from the infeasibility, but since all pilots have already received their flight plan, we want them to still flight their previously assigned flights.

The list of pilots and co-pilots on strike is given on a `csv` file:

In [16]:
strike = pd.read_csv("strike.csv")
strike[:5]

,Pilots
0,Frieda Robscheit-Robbins
1,Geraldine Seydoux
2,Gertrude B. Elion
3,Jacqueline K. Barton
4,Jocelyn Bell Burnell


We can use the same model and impose:
  1) to avoid the assignment of flights to pilots on strike
  2) to fix previously generated assignments in such a way that they are not changed
  
This can be done by fixing variables:

In [17]:
pilots_on_strike = strike['Pilots'].tolist()
for (p, v), var in instance.x.iteritems():
    if p in pilots_on_strike:
        var.value, var.fixed = 0, True
            
for (c, v), var in instance.y.iteritems():
    if c in pilots_on_strike:
        var.value, var.fixed = 0, True

for (p, v), var in instance.x.iteritems():
    if var.value > 0:
        var.fixed = True
            
for (c, v), var in instance.y.iteritems():
    if var.value > 0:
        var.fixed = True

We then solve again the problem and provide the updated total cost and the new plan:

In [18]:
results = solver.solve(instance)
print("Weekly plan cost {}".format(pyopt.value(instance.z)))

Weekly plan cost 1664.500000


In [19]:
pd.DataFrame.from_items(
    [(v, [p,c]) for p in instance.P for c in instance.C for v in instance.V 
                if instance.x[p,v].value > 0 and instance.y[c,v].value > 0 ], 
      orient='index', columns=['Pilot', 'Co-pilot'])

,Pilot,Co-pilot
ZT5269,Sir Isaac Newton,Cecilia Payne-Gaposchkin
ZT6743,Sir Isaac Newton,Flossie Wong-Staal
ZT2898,Lord Kelvin,Chien-Shiung Wu
ZT3821,Lord Kelvin,Chien-Shiung Wu
ZT5845,Blaise Pascal,Mildred S. Dresselhaus
ZT4311,Blaise Pascal,Edmond Halley
ZT6742,Sir Ernest Rutherford,Shirley Ann Jackson
ZT7579,Albert Einstein,Wilhelm Conrad Roentgen
ZT3554,Lise Meitner,Ruzena Bajcsy
ZT9968,Lise Meitner,Wilhelm Conrad Roentgen
